In [ ]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import requests
import PIL
import io

In [ ]:
def get_tag_image(tag_id: int):
    url = f"https://raw.githubusercontent.com/AprilRobotics/apriltag-imgs/master/tagStandard41h12/tag41_12_{tag_id:05d}.png"
    result = requests.get(url)
    assert result.status_code == 200
    return PIL.Image.open(io.BytesIO(result.content))

In [ ]:

def add_tag_to_canvas(c: canvas.Canvas, tag_id: int, pagesize: tuple[float, float]):
    width, height = pagesize
    img = ImageReader(get_tag_image(tag_id))
    
    tag_max_y = 0.48 * height
    tag_min_y = 0.04 * height
    tag_size = tag_max_y - tag_min_y
    
    tag_location = (0.5 * (width  - tag_size), tag_min_y)
    text_location = (width / 2.0, height * 0.02)
    c.drawCentredString(*text_location, f'Tag Id {tag_id:05d}')
    c.drawImage(img, *tag_location, width=tag_size, height=tag_size)
    
    c.line(width * 0.1, height * 0.5, width*0.9, height*0.5)
    
    c.rotate(180)
    text_location = (-width / 2.0, -height * 0.98)
    tag_location = (-0.5 * (width + tag_size), -height + tag_min_y)
    c.drawCentredString(*text_location, f'Tag Id {tag_id:05d}')
    c.drawImage(img, *tag_location, width=tag_size, height=tag_size)
    c.showPage()


In [ ]:

file_name = "/tmp/tags.pdf"
c = canvas.Canvas(file_name, pagesize=letter)

for tag_id in range(200, 230):
    add_tag_to_canvas(c, tag_id, letter)
c.save()